In [12]:
from keras.datasets import mnist  
from keras.utils import np_utils  
import numpy as np  
np.random.seed(10)  
  
# Read MNIST data  
(X_Train, y_Train), (X_Test, y_Test) = mnist.load_data()

y_Test.shape




(10000,)

In [4]:



#載入大小為60000張 28 * 28大小的圖片 且像素值介於0~255 標準化後在0~1
  
# Translation of data  
X_Train40 = X_Train.reshape(X_Train.shape[0], 28, 28, 1).astype('float32')


#因為CNN有捲積與池化運算,必須保留影像的維度.因此將資料轉成28 * 28 * 1(高)的影像單位


X_Test40 = X_Test.reshape(X_Test.shape[0], 28, 28, 1).astype('float32')  


In [13]:
# Standardize feature data

#標準化即是將原本的數據作線性轉化, 目的是要將圖片灰度與曝光度給忽略 ,讓電腦不會因此
# 辨識錯誤
# https://zhuanlan.zhihu.com/p/35597976
#標準化可以使圖片映射至同一個特定區間,讓CNN可以對數據進行同樣的處理
#就像....同一起跑點的感覺


X_Train40_norm = X_Train40 / 255  
X_Test40_norm = X_Test40 / 255  

# 為何除255? 
# 1個點像素值為0~255 , 標準化後數值降低 , ???



  
# Label Onehot-encoding

# one hot 編碼 : 機器學習中,特徵有時並不是連續值,有可能是分類值(像是男,女而不是123)

# 對這類特徵 我們需要將其數字化才能放入學習

# 就使用到 one hot encoding 

# 已知運作 還不知為何用了就能丟進去學習

# https://blog.csdn.net/taotiezhengfeng/article/details/73692239

#簡單來說就是 將各個特徵值標示為1而不是2 3 4等等
# 是讓最初訓練時每個特徵間的距離都相同而做的

# 

# https://www.itread01.com/content/1501748526.html
y_TrainOneHot = np_utils.to_categorical(y_Train)  
print(y_Train[0])
print(y_TrainOneHot.shape)
y_TestOneHot = np_utils.to_categorical(y_Test)
print(y_Test[0])

5
(60000, 10)
7


In [4]:
from keras.models import Sequential  
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D  
from keras.models import Model
#  從Keras呼叫Conv2D ,與 Maxpooling2D
# 先卷積:
from keras.layers import Input, Dense
from keras.models import Model


# This creates a model that includes
# the Input layer and three Dense layers
model = Sequential()


# Create CN layer 1  
model.add(Conv2D(filters=16,#filter 產生16個隨機的小方塊
                 kernel_size=(5,5), #卷積方塊大小為5 * 5 = 25
                 
                 
                 padding='same',  #將卷積後縮小的feature map 補上0 就像畫框一樣
                 
# 又被稱作zero-padding
#                  為何要補??
                 
#                  因為他能夠抑制圖像邊緣的效果，
#                  一般情況下我們認為圖像的中間部分比起周圍更重要。
                 
                 
                 input_shape=(28,28,1),  
                 activation='relu'))  #使用激活函數主要是引入表達能力(非線性)
                                        # 之後研究
    
# 再池化:


# Create Max-Pool 1  
model.add(MaxPooling2D(pool_size=(2,2)))

#pooling是保存重要特徵並減少feature map 的大小

# 藉由減少大小可加快運作效率 也能避免過擬合的情況發生

  
# Create CN layer 2  
model.add(Conv2D(filters=36,  
                 kernel_size=(5,5),  
                 padding='same',  
                 input_shape=(28,28,1),  
                 activation='relu'))  
  
# Create Max-Pool 2  
model.add(MaxPooling2D(pool_size=(2,2)))  
  
# Add Dropout layer  
# dropput 函式可防止過擬合, 輸入參數則可丟棄一部份的檔案?
# 首要參數rate為關掉隱藏層節點的比例，
# 利用隨機關掉隱藏層節點與輸入神經元的連結，不更新權重(W)，
# 造成多個結果，再作比較去除極端值，即可達到避免過度擬合的現象。

model.add(Dropout(0.25))  


# 建立神經網路

model.add(Flatten())


# flatten : 將前面的東西拉平後 , 給NN做訓練

# Hidden layer 

# 添加hidden layer 在中間

# 主要功能為增加神經網路的複雜度,模擬非線性關系,不過太多會導致过擬合

# dense即為常用的全連結層

# 全連結層的作用即是分類

model.add(Dense(128, activation='relu'))
# units 代表的是維度 


# 128個特徵點 每個特徵點都向之前卷積池化後的feature map 取得不同的權重



model.add(Dropout(0.5)) 


# 建立輸出層
model.add(Dense(10, activation='softmax')) 
# 使用softmax 可將輸出轉成每個數字的機率


In [2]:
# 查看模型
model.summary()  
print("") 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               225920    
__________

In [6]:
# 模型定義好了 接著要開始訓練

# 1.定義訓練
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 選擇損失函數 , 優化的方法 ,成效衡量方式

# * loss: 設定 Loss Function, 這邊選定 Cross Entropy 作為 Loss Function.
# * optimizer: 設定訓練時的優化方法, 在深度學習使用 adam (Adam: A Method for Stochastic Optimization) 可以更快收斂, 並提高準確率.
# * metrics: 設定評估模型的方式是 accuracy 準確率.

# 2.開始訓練

train_history = model.fit(x=X_Train40_norm,  
                          y=y_TrainOneHot, validation_split=0.2,  
                          epochs=10, batch_size=480, verbose=1)  

# x : 數據
# y : 標籤

# validation_split : 用來指定訓練集的一定比例數據做為驗證集 

# epochs : 訓練在到達該值(10)的時候停止 ,沒設定就是訓練到底

# batch_size : 每次訓練的資料筆數

# verbose : 1 為 顯示epoch紀錄






Train on 48000 samples, validate on 12000 samples
Epoch 1/10
19200/48000 [===========>..................] - ETA: 18s - loss: 2.2137 - acc: 0.2401

KeyboardInterrupt: 